# API de MLflow

Esta tarea pretende poner en práctica la habilidad del estudiante para registrar un modelo sencillo en MLflow. Para ello, será necesario tener un entorno virtual con MLflow y su interfaz de usuario. El estudiante realizará los pasos solicitados en cada parte.  

En esta práctica aprenderemos a utilizar la API de MLflow (_mlflow.Client_). Esto será de mucha utilidad para realizar operaciones de forma remota, como registrar modelos, descargar modelos, desplegar modelos, etc. Aquí tenemos la documentación por si pueda ser de ayuda: https://mlflow.org/docs/latest/getting-started/logging-first-model/step2-mlflow-client.html

Todo el código que realicéis debe estar en las celdas que pone `### CÓDIGO DEL ESTUDIANTE`. Si necesitáis importar librerías, podéis hacerlo con total libertad. Pero no os salgáis del entorno virtual del curso instalando nuevas librerías adicionales. También leeréis algunas celdas que pone `### PANTALLAZO DEL ESTUDIANTE`, que aplicará poner el pantallazo y se verá en el PDF que creéis.

Una vez finalizada la actividad, guarda tu fichero y asegúrate de que no se lanza ningún error. No se corregirá ningún fichero que tenga errores de sintaxis y no se pueda, como mínimo, ejecutar completamente. RECUERDA SUBIR CADA UNO DE LOS FICHEROS .ipynb TAL CUAL (sueltos), SIN COMPRIMIR Y SIN CAMBIARLES EL NOMBRE. Los ficheros subidos deben tener exactamente el mismo nombre de fichero que tenían cuando los recibiste. Adicionalmente, subiréis un PDF o HTML con el resultado de todo el Notebook, incluidos pantallazos si se piden. La plataforma ya los separa automáticamente en carpetas que traen el nombre y apellidos del estudiante, por lo que NO es necesario que lo pongas en ninguna parte.

## Parte 1

En esta primera parte, realizaréis un modelo muy básico con la librería de `sklearn` con las siguientes características:

- el dataset será el de iris de la librería sklearn.
- se pide un train-test-split de un 30% del dataset en el conjunto de test.
- ajustaréis un modelo de _Logistic Regression_ con el conjunto de train.
- realizaréis las predicciones con el conjunto de test.
- mostraréis el _accuracy_ final con un simple print.
- utilizar la semilla 99, tanto para el train-test-split como para la Logistic Regression. 

Nota, aquí no se pide utilizar MLflow.

In [1]:
# Librerias
from utils import read_yaml
from datetime import datetime
import mlflow
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from mlflow import models

In [2]:
### CÓDIGO DEL ESTUDIANTE
def read_data_iris(test_size: float,seed: float):
    dataset = load_iris()
    x = dataset.data
    y = dataset.target
    return train_test_split(x,y,test_size = test_size, random_state = seed)

In [3]:
### CÓDIGO DEL ESTUDIANTE
def modeling(x_train: float, x_test: float, y_train: float, y_test: float, seed: int):
    model = LogisticRegression(random_state = seed)
    model.fit(x_train,y_train)
    score = model.score(x_test,y_test)

    return model, score

In [4]:
### CÓDIGO DEL ESTUDIANTE
config = read_yaml()
seed = config['ConfigLogisticRegression']['seed']
test_size = config['ConfigLogisticRegression']['test_size']

x_train, x_test, y_train, y_test = read_data_iris(test_size = test_size, seed = seed)
model, score = modeling(x_train, x_test, y_train, y_test, seed)

y_pred = model.predict(x_test)

In [5]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9555555555555556


## Parte 2

Ahora vamos a trabajar con MLflow. Tendréis que investigar cómo loggar el modelo y la métrica. Deberéis comprobar que se ve el modelo y la métrica en la interfaz de usuario. El modelo tendrá el nombre **IrisApellido1Apellido2**, donde _Apellido1_ y _Apellido2_ serán vuestros respectivos apellidos.

Adicionalmente, crearéis un cliente de MLflow con el siguiente comando `client = MlflowClient()`. Y sobre este cliente, tendréis que investigar cómo **registrar el modelo** sin usar la interfaz de usuario. Directamente a través del cliente.

Añadiréis un pantallazo al Notebook actual para demostrar que se ha hecho todo correctamente. En el pantallazo deberá aparecer la UI de MLflow con el modelo Registrado.

#### Para el primer paso tan solo he ejecutado el Logistic Regression y he abierto la interfaz de mlflow
#### con el comando en powershell mlflow ui --port 8080
#### Automaticamente se abre la interfaz de mlflow y se ven los experimentdos generado con mi nombre

In [13]:
### CÓDIGO DEL ESTUDIANTE

client = mlflow.MlflowClient()
mlflow.set_experiment(f"experiment_{config['ConfigLogisticRegression']['model_name']}")
mlflow.sklearn.autolog()
with mlflow.start_run() as run:
    model = LogisticRegression(random_state = seed)
    model.fit(x_train,y_train)
    score = model.score(x_test, y_test)
    mlflow.log_metric("score", score)
    print(f"Score del modelo: {score}")

    # registrar el modelo
    signature = models.infer_signature(x_test,y_pred)
    mlflow.sklearn.log_model( 
        sk_model= model,
        artifact_path= "sklearn-model",
        signature = signature,
        registered_model_name = config['ConfigLogisticRegression']['model_name']
    )

Score del modelo: 0.9555555555555556


Registered model 'IrisGarciaMartinez' already exists. Creating a new version of this model...
Created version '3' of model 'IrisGarciaMartinez'.


In [14]:
### PANTALLAZO DEL ESTUDIANTE
from PIL import Image

# creating a object
im = Image.open("MlflowInterfaz.png")

im.show()


In [15]:
### PANTALLAZO DEL ESTUDIANTE
### Metrica mlflow
im = Image.open("metricaMLFLOW.png")

im.show()

## Parte 3

Ahora recuperareis la última versión del modelo que acabáis de registrar llamando Este modelo lo pondremos dentro de un decorator de Flask con la siguiente cabecera:

`
@app.route('/predict', methods=['POST'])
`

`
def predict():
`

In [19]:
### CÓDIGO DEL ESTUDIANTE
import mlflow
import mlflow.sklearn
from flask import Flask, request, jsonify


# Crear una aplicación Flask
app = Flask(__name__)

# Recuperar el modelo registrado
#model_uri = "runs:/7f74f2acc5f94f02be91f36facbbe641/IrisApellido1Apellido2"
model_uri = f"runs:{model}/sklearn-model/{signature}/{config['ConfigLogisticRegression']['model_name']}"
loaded_model = mlflow.sklearn.load_model(model_uri)

# Definir la ruta para predecir
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()  # Obtener datos de entrada en formato JSON
        features = data["features"]  # Asegúrate de que los datos de entrada coincidan con las características del modelo
        prediction = loaded_model.predict([features])[0]
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)})




MlflowException: Not a proper runs:/ URI: runs:LogisticRegression(random_state=99)/sklearn-model/inputs:   [Tensor('float64', (-1, 4))]outputs:   [Tensor('int32', (-1,))]params:   None. Runs URIs must be of the form 'runs:/<run_id>/run-relative/path/to/artifact'

# Parte 4

Ahora comprobaremos que todo funciona correctamente. Para ello, aquí levantaréis el servicio de Flask sobre el puerto 5000 de localhost y utilizaréis este trozo de código desde otro notebook para comprobar que funciona correctamente el servicio y la función `predict`.

Aquí el código que podréis utilizar:

Y aquí tendréis que poner un pantallazo con el resultado de la llamada. En el pantallazo se tendrá que ver ambos notebooks, éste y el otro, con éste corriendo con la aplicación y el otro mostrando el resultado de la llamada `POST` anterior.

In [18]:
### PANTALLAZO DEL ESTUDIANTE

In [ ]:
if __name__ == "__main__":
    app.run(host="http://localhost:8080/prediction", port=5000)  # Ejecutar la aplicación Flask en el puerto 5000